In [1]:


import numpy as np

def perform_svd_decomposition(input_matrix):
    """
    Decomposes a matrix using Singular Value Decomposition (SVD) and calculates its properties,
    including the pseudo-inverse, condition number, and reconstructed matrix.

    Parameters:
    ----------
    input_matrix : np.ndarray
        A 2D NumPy array to decompose.

    Returns:
    -------
    left_singular_vectors : np.ndarray
        Left singular vectors (U matrix) of the input matrix.
    singular_values_matrix : np.ndarray
        Diagonal matrix containing singular values.
    right_singular_vectors_transpose : np.ndarray
        Transpose of right singular vectors (V^T) of the input matrix.
    condition_num : float
        Condition number, calculated as the ratio of the largest to smallest singular values.
    pseudo_inverse_matrix : np.ndarray
        Pseudo-inverse of the input matrix.

    Raises:
    ------
    ValueError:
        If the matrix is singular and not invertible.
    """

    # Step 1: Compute transpose-product matrix = A^T * A
    transpose_product_matrix = np.dot(input_matrix.T, input_matrix)
    
    # Step 2: Perform eigen decomposition for singular values and right singular vectors
    eigenvalues, right_singular_vectors = np.linalg.eig(transpose_product_matrix)
    
    # Step 3: Sort eigenvalues and corresponding vectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    right_singular_vectors = right_singular_vectors[:, sorted_indices]
    
    # Step 4: Compute singular values from the eigenvalues
    singular_values = np.sqrt(eigenvalues)
    if singular_values[-1] == 0:
        raise ValueError("Matrix is singular and does not have an inverse.")
    
    # Step 5: Calculate left singular vectors using U = A * V / singular_values
    left_singular_vectors = input_matrix @ right_singular_vectors / singular_values
    left_singular_vectors /= np.linalg.norm(left_singular_vectors, axis=0)  # Normalize columns

    # Step 6: Create a diagonal matrix for singular values
    singular_values_matrix = np.diag(singular_values)
    
    # Step 7: Compute the pseudo-inverse of the diagonal singular value matrix
    singular_values_inverse_matrix = np.diag(1 / singular_values)
    
    # Step 8: Compute the pseudo-inverse of the input matrix
    pseudo_inverse_matrix = right_singular_vectors @ singular_values_inverse_matrix @ left_singular_vectors.T

    # Step 9: Calculate the condition number (ratio of max to min singular values)
    condition_num = singular_values[0] / singular_values[-1]

    return left_singular_vectors, singular_values_matrix, right_singular_vectors.T, condition_num, pseudo_inverse_matrix


def initialize_connectivity_matrix(size):
    """Generate the connectivity matrix that defines the relationships between springs and masses."""
    connectivity_matrix = np.zeros(size)
    np.fill_diagonal(connectivity_matrix, 1)
    if size[0] > 1:
        np.fill_diagonal(connectivity_matrix[1:], -1)
    return connectivity_matrix

def generate_spring_constants_matrix(spring_constants):
    """Generate a diagonal matrix for spring constants."""
    return np.diag(spring_constants)

def calculate_gravity_forces(mass_list):
    """Compute the force vector based on mass and gravity."""
    return np.array(mass_list) * -9.81

def determine_stiffness_matrix(connectivity_matrix, spring_constants_matrix):
    """Compute the stiffness matrix from the connectivity matrix and spring constants."""
    return connectivity_matrix.T @ spring_constants_matrix @ connectivity_matrix

def solve_for_displacements(stiffness_matrix, force_vector):
    """Calculate displacements using the pseudo-inverse of the stiffness matrix."""
    _, _, _, _, inverse_stiffness_matrix = perform_svd_decomposition(stiffness_matrix)
    return inverse_stiffness_matrix @ force_vector

def compute_spring_elongation(connectivity_matrix, displacements):
    """Determine elongations of springs from displacements."""
    return connectivity_matrix @ displacements

def compute_spring_forces(spring_constants_matrix, elongation):
    """Calculate internal forces in springs based on elongation."""
    return spring_constants_matrix @ elongation

def get_user_input():
    """Gather the number of springs, masses, spring constants, and masses from the user."""
    while True:
        try:
            num_springs = int(input("Enter the number of springs:\n"))
            num_masses = int(input("Enter the number of masses:\n"))
            boundary_type = input("Enter boundary condition (1 for Fixed-Free, 2 for Fixed-Fixed):\n")
            if boundary_type == "1":
                boundary = "Fixed-Free"
            elif boundary_type == "2":
                boundary = "Fixed-Fixed"
            else:
                print("Invalid input. Choose 1 or 2.")
                continue

            if num_springs > 0 and num_masses > 0 and (
                (num_springs == num_masses and boundary == "Fixed-Free") or
                (num_springs == num_masses + 1 and boundary == "Fixed-Fixed")
            ):
                break
            else:
                print("Invalid configuration. Adjust your inputs.")
        except ValueError as e:
            print(f"Error: {e}\nEnter valid positive integers.")

    spring_constants = []
    for i in range(num_springs):
        while True:
            try:
                k = float(input(f"Enter the spring constant for spring {i+1}:\n"))
                if k > 0:
                    spring_constants.append(k)
                    break
                else:
                    print("Positive values only.")
            except ValueError as e:
                print(f"Error: {e}\nProvide a valid number.")

    masses = []
    for i in range(num_masses):
        while True:
            try:
                m = float(input(f"Enter the mass for mass {i+1}:\n"))
                if m > 0:
                    masses.append(m)
                    break
                else:
                    print("Positive values only.")
            except ValueError as e:
                print(f"Error: {e}\nProvide a valid number.")

    return num_springs, num_masses, spring_constants, masses


def main():
    num_springs, num_masses, spring_constants, masses = get_user_input()
    connectivity_matrix_size = (num_springs, num_masses)

    connectivity_matrix = initialize_connectivity_matrix(connectivity_matrix_size)
    spring_constants_matrix = generate_spring_constants_matrix(spring_constants)
    force_vector = calculate_gravity_forces(masses)
    stiffness_matrix = determine_stiffness_matrix(connectivity_matrix, spring_constants_matrix)

    displacements = solve_for_displacements(stiffness_matrix, force_vector)
    elongations = compute_spring_elongation(connectivity_matrix, displacements)
    internal_spring_forces = compute_spring_forces(spring_constants_matrix, elongations)

    print(f"\nConnectivity Matrix A:\n{connectivity_matrix}")
    print(f"\nSpring Constant Matrix C:\n{spring_constants_matrix}")
    print(f"\nForce Vector:\n{force_vector}")
    print(f"\nStiffness Matrix K:\n{stiffness_matrix}")
    print(f"\nDisplacement Vector:\n{displacements}")
    print(f"\nSpring Elongations:\n{elongations}")
    print(f"\nInternal Spring Forces:\n{internal_spring_forces}")


if __name__ == "__main__":
    main()


Enter the number of springs:
3
Enter the number of masses:
6
Enter boundary condition (1 for Fixed-Free, 2 for Fixed-Fixed):
2
Invalid configuration. Adjust your inputs.
Enter the number of springs:
2
Enter the number of masses:
2
Enter boundary condition (1 for Fixed-Free, 2 for Fixed-Fixed):
2
Invalid configuration. Adjust your inputs.
Enter the number of springs:
2
Enter the number of masses:
2
Enter boundary condition (1 for Fixed-Free, 2 for Fixed-Fixed):
1
Enter the spring constant for spring 1:
2
Enter the spring constant for spring 2:
2
Enter the mass for mass 1:
2
Enter the mass for mass 2:
2

Connectivity Matrix A:
[[ 1.  0.]
 [-1.  1.]]

Spring Constant Matrix C:
[[2. 0.]
 [0. 2.]]

Force Vector:
[-19.62 -19.62]

Stiffness Matrix K:
[[ 4. -2.]
 [-2.  2.]]

Displacement Vector:
[-19.62 -29.43]

Spring Elongations:
[-19.62  -9.81]

Internal Spring Forces:
[-39.24 -19.62]
